In [1]:
import numpy as np
import pickle 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable, Function

In [2]:
from sklearn.datasets import make_moons, make_blobs
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [3]:
train_size=2500
test_size=500

In [4]:
data_path = '/home/tako/saerom/data/'

In [5]:
src='books'
tgt='electronics'

In [6]:
f = open(data_path+'doc2vec'+'_source_'+src+'_target_'+tgt+'_dbow_data.pickle','rb')
aa = pickle.load(f)
f.close()

In [9]:
Xs_train = aa['docvec'][aa['st_label']==1][:train_size]
ys_train = (aa['true_label'][aa['st_label']==1][:train_size] + 1)/2
Xt_train = aa['docvec'][aa['st_label']==0][:train_size]
yt_train = (aa['true_label'][aa['st_label']==0][:train_size] + 1)/2
Xs_test = aa['docvec'][aa['st_label']==1][-test_size:]
ys_test = (aa['true_label'][aa['st_label']==1][-test_size:] +1 )/2
Xt_test = aa['docvec'][aa['st_label']==0][-test_size:]
yt_test = (aa['true_label'][aa['st_label']==0][-test_size:] +1 )/2

In [10]:
imput_dim = Xs_train.shape[1]
hidden_dim =200
feature_dim =100
classnum=1

learning_rate = 1e-2          # learning rate
num_epochs = 100               # number of epochs to train models
batch_size = 100               # size of image sample per epoch 

In [11]:
# load source domain dataset

In [12]:
def eval_func(Xt_test, yt_test):
    out = c_clf(f_ext(Variable(torch.from_numpy(Xt_test[:len(Xt_test)]).float())).view(len(Xt_test), -1))
    right=0
    for i in range(len(out)):
        if out.data.cpu().numpy()[i] - yt_test[i]<0.5:
           right += 1
    return right/len(Xt_test)

In [13]:
#batch

In [14]:
def batch_generator(batch_size, data, labels):
    size = data.shape[0]
    idx_array = np.arange(size)
    n_batch = int(np.ceil(size / float(batch_size)))
    batches = [(int(i * batch_size), int(min(size, (i + 1) * batch_size))) for i in range(n_batch)]
    for batch_index, (start, end) in enumerate(batches):
        print('\rBatch {}/{}'.format(batch_index+1, n_batch), end='')
        batch_ids = idx_array[start:end]
        if labels is not None:
            yield Variable(torch.from_numpy(data[batch_ids])), Variable(torch.from_numpy(labels[batch_ids])), batch_ids
        else:
            yield Variable(torch.from_numpy(data[batch_ids])), batch_ids

In [15]:
def eval_clf(model1, model2, x, y, n):
    out = model1(model2(Variable(torch.from_numpy(x[:n]).float())).view(n, -1))
    preds = out.max(1)[1]
    return accuracy_score(y_true=[np.argmax(i) for i in y[:n]], y_pred=preds.data.numpy().ravel())


In [16]:
#DANN MODEL

In [17]:
class GradReverse(Function):
    def __init__(self, lambd):
        self.lambd = lambd

    def forward(self, x):
        return x.view_as(x)

    def backward(self, grad_output):
        return (grad_output * -self.lambd)

def grad_reverse(x, lambd):
    return GradReverse(lambd)(x)

In [18]:
class feature_extract(nn.Module):
    def __init__(self, imput_dim, feature_dim):
        super(feature_extract, self).__init__()
        self.fc1 = nn.Linear(imput_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, feature_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [19]:
class dann_domain_clf(nn.Module):
    def __init__(self):
        super(dann_domain_clf, self).__init__()
        self.fc1 = nn.Linear(feature_dim, 100) 
        self.fc2 = nn.Linear(100, 1)
        self.drop = nn.Dropout2d(0.25)

    def set_lambda(self, lambd):
        self.lambd = lambd
        
    def forward(self, x):
        x = grad_reverse(x, self.lambd)
        x = F.leaky_relu(self.drop(self.fc1(x)))
        x = self.fc2(x)
        return F.sigmoid(x)

In [20]:
class dann_class_clf(nn.Module):
    def __init__(self):
        super(dann_class_clf, self).__init__()
        self.fc1 = nn.Linear(feature_dim, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, classnum)
        self.drop = nn.Dropout2d(0.2)
        
    def forward(self, x):
        x = F.relu(self.drop(self.fc1(x)))
        x = F.relu(self.drop(self.fc2(x)))
        x = self.fc3(x)
        return F.sigmoid(x)

In [21]:
f_ext, d_clf, c_clf = feature_extract(imput_dim, feature_dim), dann_domain_clf(), dann_class_clf()

In [22]:
#loss function

In [23]:
d_crit = nn.BCELoss()
c_crit = nn.BCELoss()

In [24]:
d_optimizer = optim.SGD(d_clf.parameters(), lr=learning_rate, momentum=0.9)
c_optimizer = optim.SGD(c_clf.parameters(), lr=learning_rate, momentum=0.9)
f_optimizer = optim.SGD(f_ext.parameters(), lr=learning_rate, momentum=0.9)

In [25]:
num_steps = num_epochs * (Xs_train.shape[0] / batch_size)
yd = Variable(torch.from_numpy(np.hstack([np.repeat(1, int(batch_size / 2)), np.repeat(0, int(batch_size / 2))]).reshape(batch_size, 1)))
j = 0

In [26]:
# train DANN model
print('Training DANN model..')
for i in range(num_epochs):
    source_gen = batch_generator(int(batch_size / 2), Xs_train, ys_train)
    target_gen = batch_generator(int(batch_size / 2), Xt_train, None)

    # iterate over batches
    for (xs, ys, _) in source_gen:
        p = float(j) / num_steps
        lambd = round(2. / (1. + np.exp(-10. * p)) - 1, 3)
        lr = 0.01 / (1. + 100 * p)**0.75
        d_clf.set_lambda(lambd)
        d_optimizer.lr = lr
        c_optimizer.lr = lr
        f_optimizer.lr = lr
        j += 1
        
        # get next target batch
        xt, _ = next(target_gen)

        # exit when batch size mismatch
        if len(xs) + len(xt) != batch_size:
            print('aa')
            continue
        
        # concatenate source and target batch
        x = torch.cat([xs, xt], 0)
        
        # 1) train feature_extractor and class_classifier on source batch
        # reset gradients
        f_ext.zero_grad()
        c_clf.zero_grad()
        
        # calculate class_classifier predictions on batch xs
        c_out = c_clf(f_ext(xs.float()).view(int(batch_size / 2), -1))

        # optimize feature_extractor and class_classifier on output
        f_c_loss = c_crit(c_out, ys.float())
        f_c_loss.backward(retain_graph = True)
        c_optimizer.step()
        f_optimizer.step()

        # 2) train feature_extractor and domain_classifier on full batch x
        # reset gradients
        f_ext.zero_grad()
        d_clf.zero_grad()
        
        # calculate domain_classifier predictions on batch x
        f_ext(x.float())
        d_out = d_clf(f_ext(x.float()).view(batch_size, -1))
        
        # use normal gradients to optimize domain_classifier
        f_d_loss = d_crit(d_out, yd.float())
        f_d_loss.backward(retain_graph = True)
        d_optimizer.step()
        f_optimizer.step()
        
        # print batch statistics
        print('\rEpoch         - d_loss: {} - c_loss: {}'.format(format(f_d_loss.data[0], '.4f'),
                            format(f_c_loss.data[0], '.4f')), end='')           
    
    # print epoch statistics
    t_acc = eval_func(Xt_train, yt_train)
    s_acc = eval_func(Xs_train, ys_train)
    #t_acc = eval_clf(c_clf, f_ext, Xt_test, yt_test, Xt_test[0])
    #s_acc = eval_clf(c_clf, f_ext, Xs_test, ys_test, Xs_test[0])
    print(' - target_acc: {} - source_acc: {}'.format(format(t_acc, '.4f'), format(s_acc, '.4f')))

Training DANN model..
Batch 3/50

/home/tako/anaconda3/envs/pytorch2/lib/python3.6/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch         - d_loss: 0.6932 - c_loss: 0.6950 - target_acc: 0.5140 - source_acc: 0.5300
Epoch         - d_loss: 0.6939 - c_loss: 0.6955 - target_acc: 0.5124 - source_acc: 0.5228
Epoch         - d_loss: 0.6923 - c_loss: 0.6950 - target_acc: 0.5132 - source_acc: 0.5204
Epoch         - d_loss: 0.6923 - c_loss: 0.6930 - target_acc: 0.5028 - source_acc: 0.5228
Epoch         - d_loss: 0.6943 - c_loss: 0.6958 - target_acc: 0.5172 - source_acc: 0.5300
Epoch         - d_loss: 0.6939 - c_loss: 0.6953 - target_acc: 0.5320 - source_acc: 0.5472
Epoch         - d_loss: 0.6930 - c_loss: 0.6933 - target_acc: 0.5356 - source_acc: 0.5588
Epoch         - d_loss: 0.6924 - c_loss: 0.6938 - target_acc: 0.5464 - source_acc: 0.5732
Epoch         - d_loss: 0.6949 - c_loss: 0.6915 - target_acc: 0.5504 - source_acc: 0.5996
Epoch         - d_loss: 0.6927 - c_loss: 0.6867 - target_acc: 0.5904 - source_acc: 0.6540
Epoch         - d_loss: 0.6915 - c_loss: 0.6833 - target_acc: 0.6400 - source_acc: 0.7188
Epoch     

In [7]:
Xs_train = aa['docvec'][aa['st_label']==0][:train_size]
ys_train = (aa['true_label'][aa['st_label']==0][:train_size] + 1)/2
Xt_train = aa['docvec'][aa['st_label']==1][:train_size]
yt_train = (aa['true_label'][aa['st_label']==1][:train_size] + 1)/2
Xs_test = aa['docvec'][aa['st_label']==0][-test_size:]
ys_test = (aa['true_label'][aa['st_label']==0][-test_size:] +1 )/2
Xt_test = aa['docvec'][aa['st_label']==1][-test_size:]
yt_test = (aa['true_label'][aa['st_label']==1][-test_size:] +1 )/2